In [1]:
import pandas as pd
import numpy as np

In [12]:
%%time
import funcs

rats_gdo = funcs.get_rats_gdo()
rats_gdo.head(3)

CPU times: user 2.71 s, sys: 74.5 ms, total: 2.78 s
Wall time: 6.48 s


,NAT.CODIGO,NAT.DESCRICAO,DTA_HRA_INCLUSAO,DTA_INICIO,HRA_INICIO,DTA_TERMINO,HRA_TERMINO,DES_ALVO_EVENTO,DES_LUGAR,NOM_OPERACAO,...,CIA,VIATURAS,EFETIVO,EFETIVIDADE_PARCIAL,Qde de locais fiscalizados,Qde de pessoas abordadas,Qde de pessoas que sopraram o etilometro,Qde de veiculos fiscalizados,EFICIENCIA_E_EFICACIA,EFETIVIDADE
RAT.NUM_ATIVIDADE,,,,,,,,,,,,,,,,,,,,,
2018-046472984-001,Y07001,OPERACAO DE BATIDA POLICIAL,17/10/2018,17/10/2018,18:43,17/10/2018,19:00,TRANSEUNTE,VIA DE ACESSO PUBLICA,,...,139 CIA,1,2,0,0,0,0,0,0,0
2018-051709149-001,Y07001,OPERACAO DE BATIDA POLICIAL,20/11/2018,20/11/2018,05:25,20/11/2018,07:15,TRANSEUNTE,METALURGICA / SIDERURGICA,,...,142 CIA,1,2,0,0,0,0,0,0,0
2018-052156427-001,Y07001,OPERACAO DE BATIDA POLICIAL,22/11/2018,22/11/2018,20:20,22/11/2018,20:45,TRANSEUNTE,VIA DE ACESSO PUBLICA,,...,142 CIA,1,2,0,0,0,0,0,0,0


<h2 style='color:#009'>Código para gerar relatório de LEI SECA, por Cias, por ano, por mês</h2>

In [21]:
rats_gdo[
    ( rats_gdo['MES'] == 5 )
    & ( rats_gdo['NAT.CODIGO'] == 'Y04012' )
    & ( rats_gdo['ANO'] == 2020 )
    & ( rats_gdo['MUNICIPIO'] == 'DIVINOPOLIS' )
    & ~(rats_gdo['NOM_UNID_RESPONSAVEL'].str.contains('PRV'))
][[
    'SETOR', 'CIA', 'NOM_UNID_RESPONSAVEL',
    'VIATURAS',
    'EFETIVO', 'EFETIVIDADE_PARCIAL', 'Qde de locais fiscalizados',
    'Qde de pessoas abordadas', 'Qde de pessoas que sopraram o etilometro',
    'Qde de veiculos fiscalizados', 'EFICIENCIA_E_EFICACIA', 'EFETIVIDADE'
]]

,SETOR,CIA,NOM_UNID_RESPONSAVEL,VIATURAS,EFETIVO,EFETIVIDADE_PARCIAL,Qde de locais fiscalizados,Qde de pessoas abordadas,Qde de pessoas que sopraram o etilometro,Qde de veiculos fiscalizados,EFICIENCIA_E_EFICACIA,EFETIVIDADE
RAT.NUM_ATIVIDADE,,,,,,,,,,,,
2020-020735210-001,HIPER CENTRO,53 CIA,240 CIA TM/23 BPM/7 RPM,1,2,0,0,51,0,47,1,0
2020-020746314-001,HIPER CENTRO,53 CIA,240 CIA TM/23 BPM/7 RPM,1,2,1,0,46,6,52,1,1
2020-021777510-001,PORTO VELHO,142 CIA,240 CIA TM/23 BPM/7 RPM,1,2,1,0,9,1,5,1,1
2020-021985822-001,SAO JOSE,139 CIA,240 CIA TM/23 BPM/7 RPM,1,2,1,0,80,0,60,1,1
2020-023224000-001,HIPER CENTRO,53 CIA,240 CIA TM/23 BPM/7 RPM,1,2,0,0,60,0,50,1,0
2020-023304978-001,HIPER CENTRO,53 CIA,240 CIA TM/23 BPM/7 RPM,1,2,1,0,50,2,0,1,1
2020-023992411-001,NITEROI,142 CIA,142 CIA PM/23 BPM/7 RPM,3,4,1,0,8,0,12,1,1
2020-024232424-001,PORTO VELHO,142 CIA,142 CIA PM/23 BPM/7 RPM,2,2,1,0,15,0,7,1,1
2020-024263375-001,PORTO VELHO,142 CIA,240 CIA TM/23 BPM/7 RPM,1,3,1,0,10,0,4,1,1


In [8]:
rats_gdo.columns

Index(['NAT.CODIGO', 'NAT.DESCRICAO', 'DTA_HRA_INCLUSAO', 'DTA_INICIO',
       'HRA_INICIO', 'DTA_TERMINO', 'HRA_TERMINO', 'DES_ALVO_EVENTO',
       'DES_LUGAR', 'NOM_OPERACAO', 'COD_UNIDADE_SERVICO',
       'NOM_UNID_RESPONSAVEL', 'TIPO_LOGRADOURO', 'LOGRADOURO', 'DES_ENDERECO',
       'NUM_ENDERECO', 'COMPLEMENTO_ALFA', 'COMPLEMENTO_ENDERECO',
       'NUM_COMPLEMENTAR', 'COD_BAIRRO', 'NOME_BAIRRO', 'TIPO_LOGRADOURO2',
       'LOGRADOURO2', 'DES_ENDERECO2', 'COD_MUNICIPIO', 'MUNICIPIO',
       'LATITUDE', 'LONGITUDE', 'COD_UNIDADE_AREA', 'NOM_UNIDADE_AREA',
       'DIGITADOR', 'DTA_HRA_INICIO_DT', 'DTA_HRA_TERMINO_DT', 'TEMPO_DT',
       'TEMPO_INT', 'DIA', 'MES', 'ANO', 'DEZENA', 'SETOR', 'CIA', 'VIATURAS',
       'EFETIVO', 'EFETIVIDADE_PARCIAL', 'Qde de locais fiscalizados',
       'Qde de pessoas abordadas', 'Qde de pessoas que sopraram o etilometro',
       'Qde de veiculos fiscalizados', 'EFICIENCIA_E_EFICACIA', 'EFETIVIDADE'],
      dtype='object')

In [8]:
rats_ols = rats_gdo[        
    (~rats_gdo['NOM_UNID_RESPONSAVEL'].str.contains('PRV'))
    & (rats_gdo['NAT.CODIGO'] == 'Y04012')
]
rats_ols.loc[:,'CIA_OLS'] = np.select([
    rats_ols.loc[:,'NOM_UNID_RESPONSAVEL'].str.contains('240')
], [
    '240 CIA'
    ], default = rats_ols.loc[:,'CIA'])

ols_by_cia = rats_ols[
    rats_ols['ANO'] >= 2020
].groupby(['ANO','MES','CIA_OLS']).sum()['EFETIVIDADE'].unstack(['MES']).fillna(0)

ols_by_cia.loc[:,'ACUM'] = ols_by_cia.sum(1)
ols_by_cia.loc[(2020, '23 BPM'),:] = ols_by_cia.sum()

# rats_ols

ols_by_cia = ols_by_cia.astype('int16')
ols_by_cia.columns =  pd.MultiIndex.from_product([['OLS'], ols_by_cia.columns])
ols_by_cia

/home/seo23bpm/.asdf/installs/python/anaconda3-2019.10/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/seo23bpm/.asdf/installs/python/anaconda3-2019.10/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


OLS                   
MES            1  2  3   4   5 ACUM
ANO  CIA_OLS                       
2020 139 CIA   1  0  1   2   0    4
     142 CIA   0  1  0   0   4    5
     240 CIA   0  1  2   6   6   15
     51 CIA    0  3  1   2   2    8
     53 CIA    2  0  0   1   0    3
     23 BPM    3  5  4  11  12   35

<h2 style='color:#009'>Código para gerar relatório de RQV, por setores, por ano, por mês</h2>

In [13]:
rats_rqv = rats_gdo[
    (rats_gdo['NAT.CODIGO'].str.contains('Y0700[1345]')) &
    ( ~ rats_gdo['NOM_UNID_RESPONSAVEL'].str.contains('IND PE')) &
    ( ~ rats_gdo['NOM_UNID_RESPONSAVEL'].str.contains('BPE')) &
    ( ~ rats_gdo['NOM_UNID_RESPONSAVEL'].str.contains('CPE')) &
    (rats_gdo['ANO'] == 2020)
]

rqv_by_cia = rats_rqv.groupby(['MES','CIA']).sum()[['EFICIENCIA_E_EFICACIA','EFETIVIDADE']]
rqv_by_cia = rqv_by_cia.unstack('MES')
rqv_by_cia.loc['23 BPM'] = rqv_by_cia.sum()
rqv_by_cia.loc[:,('EFICIENCIA_E_EFICACIA', 'ACUM')] = rqv_by_cia.loc[:,'EFICIENCIA_E_EFICACIA'].sum(axis=1)
rqv_by_cia.loc[:,('EFETIVIDADE', 'ACUM')] = rqv_by_cia.loc[:,'EFETIVIDADE'].sum(axis=1)
rqv_by_cia = rqv_by_cia[['EFICIENCIA_E_EFICACIA', 'EFETIVIDADE']]
rqv_by_cia.fillna(0)

EFICIENCIA_E_EFICACIA                           EFETIVIDADE            \
MES                         1    2    3    4    5  ACUM           1    2    3   
CIA                                                                             
139 CIA                   214  235  208  154  209  1020          44   40   37   
142 CIA                   416  369  337  205  181  1508         142  143  138   
51 CIA                    251  154  126  120  186   837          92   59   63   
53 CIA                    221  175  188  149  181   914          54   30   27   
23 BPM                   1102  933  859  628  757  4279         332  272  265   

                         
MES        4    5  ACUM  
CIA                      
139 CIA   34   52   207  
142 CIA   84   69   576  
51 CIA    50   97   361  
53 CIA    23   24   158  
23 BPM   191  242  1302

In [14]:
writer = pd.ExcelWriter('rqv_e_ols.xlsx', engine='xlsxwriter')
rqv_by_cia.to_excel(writer, sheet_name='RQV')
ols_by_cia.to_excel(writer, sheet_name='OLS')
rats_gdo.to_excel(writer, sheet_name='Todos os registros')
writer.save()